<img width="800px" src="../fidle/img/header.svg"></img>

# <!-- TITLE --> [DRL1] - Solving CartPole with DQN
<!-- DESC --> Using a a Deep Q-Network to play CartPole - an inverted pendulum problem (PyTorch)
<!-- AUTHOR : Nathan Cassereau (IDRIS) and Bertrand Cabot (IDRIS) -->



By Nathan Cassereau (IDRIS) and Bertrand Cabot (IDRIS)




## Objectives

* Understand the code behind the DQN algorithm
* Visualize the result for fun purposes :)

This notebook implements a DQN from scratch and trains it. It is simply a vanilla DQN with a target network (sometimes referred as Double DQN). More sophisticated and recent modifications might help stabilize the training.

Considering that we are going to use a tiny network for a simple environment, matrix multiplications are not that time consuming, and using a GPU can be detrimental as communications between CPU and GPU are no longer negligeable compared to forward and backward steps. This notebook will therefore be executed on CPU.

The chosen environment will be imported from the gym toolkit (https://gym.openai.com/).

## Demonstration steps:

- Define numerous hyperparameters
- Implement the Q-Network
- Implement an agent following the Double DQN algorithm
- Train it for a few minutes
- Visualize the result

## Installations

Gym requires a graphical interface to render a state observation. Xvfb allows to run the notebook headless. This software is not available on Jean Zay's compute node, hence the usage of Google colab.

In [ ]:
!pip3 install pyvirtualdisplay
!pip install pyglet==1.5.11
!apt-get install x11-utils > /dev/null 2>&1 
!apt-get install -y xvfb python-opengl > /dev/null 2>&1

## Imports

I chose to use Pytorch to implement this DQN due to its straightforward API and personal preferences.
Gym implements the environment.

In [ ]:
import numpy as np

import torch
import torch.nn as nn

import gym
from gym import wrappers

import random
from tqdm.notebook import tqdm

import functools
import matplotlib.pyplot as plt
import os
import io
import base64
import glob
from IPython.display import display, HTML

## Hyperparameters

The size of the replay buffer does not matter much. In this case, it is big enough to hold every transitions we will have in our training. This choice does have a huge impact on memory though.

Warm-up allows the network to gather some information before the training process begins.

The target network will only be updated once every 10k steps in order to stabilize the training.

The exploration rate is linearly decreasing, although an exponential curve is a sound and common choice as well.

As mentioned above, only the CPU will be used, the GPU would be useful for bigger networks, and / or environments which have a torch tensor internal state.

Considering this is a simple DQN implementation, its stability leaves a lot to be desired. In order not to rely on luck, a decent seed was chosen.

In [ ]:
learning_rate = 0.0001
buffer_size = 200000
warmup_steps = 10000
batch_size = 32
gamma = 0.99
train_freq = 4
target_update_interval = 10000
exploration_fraction = 0.1
exploration_initial_eps = 1.0
exploration_final_eps = 0.05
device = torch.device("cpu") # torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed = 987654321
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)

## Q-Network and Agent implementation

In [ ]:
class DQN(nn.Module):

    def __init__(self):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(4, 64)
        self.layer2 = nn.Linear(64, 64)
        self.layer3 = nn.Linear(64, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        return self.layer3(x)

    def compute_target(self, x, rewards):
        with torch.no_grad():
            values = torch.zeros(x.shape[0], device=device)
            values[rewards != 1] = torch.max(self.forward(x[rewards != 1]), dim=-1)[0]
            values = rewards + gamma * values
        return values

    def predict(self, x):
        if len(x.shape) < 2:
            x = x[None, :]
        with torch.no_grad():
            x = torch.argmax(self.forward(x), dim=-1)
        if x.device.type == "cuda":
            x = x.cpu()
        return x

class Agent:

    def __init__(self, env):
        self.env = env
        self.q_network = DQN().to(device)
        self.target_network = DQN().to(device)
        self.target_network.eval()
        self.synchronize()
        self.optimizer = torch.optim.Adam(self.q_network.parameters(), lr=learning_rate)
        self.criterion = nn.MSELoss()
        self.buffer = []
        self.n_updates = 0
    
    def add_transition(self, state, action, reward, nextState):
        self.buffer.append((state, action, reward, nextState))
        if len(self.buffer) > buffer_size:
            self.buffer.pop(random.randrange(len(self.buffer)))

    def sample(self):
        transitions = random.sample(self.buffer, batch_size)
        states, actions, rewards, nextStates = zip(*transitions)
        states = torch.stack(states).to(device)
        actions = torch.cat(actions).to(device)
        rewards = torch.cat(rewards).to(device)
        nextStates = torch.stack(nextStates).to(device)
        return states, actions, rewards, nextStates
    
    def train_step(self, step):
        if step % target_update_interval == 0:
            self.synchronize()
        if step < warmup_steps or step % train_freq != 0:
            return 0.

        states, actions, rewards, nextStates = self.sample()
        output = self.q_network(states)
        output = torch.gather(output, 1, actions.unsqueeze(-1)).view(-1)
        expectedOutput = self.target_network.compute_target(nextStates, rewards).view(-1)
        self.optimizer.zero_grad()
        loss = self.criterion(output, expectedOutput)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.q_network.parameters(), 10)
        self.optimizer.step()
        self.n_updates += 1
        return loss.item()

    def synchronize(self):
        self.target_network.load_state_dict(self.q_network.state_dict())

    def play(self, state, exploration_rate=0.):
        if random.random() > exploration_rate:
            return self.q_network.predict(state.to(device))
        else:
            shape = (state.shape[0],) if len(state.shape) > 1 else (1,)
            return torch.randint(0, 2, size=shape)

    @functools.lru_cache(maxsize=None)
    def exploration_slope(self, total_steps):
        return (exploration_initial_eps - exploration_final_eps) / (exploration_fraction * total_steps)

    def exploration(self, step, total_steps):
        eps = exploration_initial_eps - step * self.exploration_slope(total_steps)
        return max(eps, exploration_final_eps)

    def train(self, total_steps):
        obs = torch.from_numpy(env.reset()).float()

        n_episodes = 0
        length_current_episode = 0
        lengths = []
        avg_reward = 0
        loss_backup = 0.
        acc_loss = 0.
        acc_loss_count = 0
        self.rewards = []

        with tqdm(range(total_steps), desc="Training agent", unit="steps") as pbar:
            for step in pbar:
                eps = self.exploration(step, total_steps)

                action = self.play(obs, eps)
                new_obs, _, done, info = env.step(action.item())
                reward = torch.tensor([1.0 if not done else -1.0], dtype=torch.float32)
                new_obs = torch.from_numpy(new_obs).float()

                self.add_transition(obs, action, reward, new_obs)
                loss = self.train_step(step)
                if loss != 0:
                    acc_loss += loss
                    acc_loss_count += 1

                if done:
                    obs = torch.from_numpy(env.reset()).float()
                    n_episodes += 1
                    lengths.append(length_current_episode)
                    self.rewards.append(length_current_episode)
                    length_current_episode = 0
                    if len(lengths) >= 25:
                        avg_reward = sum(lengths) / len(lengths)
                        if acc_loss_count != 0:
                            loss_backup = acc_loss / acc_loss_count
                        else:
                            loss_backup = "??"
                        acc_loss = 0.
                        acc_loss_count = 0
                        lengths = []
                else:
                    obs = new_obs
                    length_current_episode += 1

                pbar.set_postfix({
                    "episodes": n_episodes,
                    "avg_reward": avg_reward,
                    "loss": loss_backup,
                    "exploration_rate": eps,
                    "n_updates": self.n_updates,
                })

## Defining the environment

In [ ]:
env = gym.make("CartPole-v1")
env.seed(seed+2)
env.reset()

## Training our agent

In [ ]:
agent = Agent(env)
agent.train(120000)

## Episodes length

A very noisy curve. It does reach satisfying levels though.

In [ ]:
fig = plt.figure(figsize=(20, 12))
plt.plot(agent.rewards)
plt.xlabel("Episodes")
plt.ylabel("Episode length")
plt.show()

## Result visualisation

In [ ]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [ ]:
env = gym.make("CartPole-v1")
env.seed(4)
env = wrappers.Monitor(env, "./CartPole-v1/", force=True)

obs = env.reset()
i = 0

while True:
    action = agent.q_network.predict(torch.from_numpy(obs).float().to(device))
    
    obs, rewards, done, info = env.step(action.item())
    env.render()
    if done:
        break
    else:
        i += 1
env.close()
print(f"Survived {i} steps")

In [ ]:
def ipython_show_video(path):
    """Shamelessly stolen from https://stackoverflow.com/a/51183488/9977878
    """
    if not os.path.isfile(path):
        raise NameError("Cannot access: {}".format(path))

    video = io.open(path, 'r+b').read()
    encoded = base64.b64encode(video)

    display(HTML(
        data="""
        <video alt="test" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        </video>
        """.format(encoded.decode('ascii'))
    ))

ipython_show_video(glob.glob("/content/CartPole-v1/*.mp4")[0])